In [2]:
import pandas as pd

# Define the path to your dataset
file_path = 'laptops.csv'  # Update this path as needed

# Load the dataset
laptop_data = pd.read_csv(file_path)

# Display the first few rows of the dataset
print(laptop_data.head())

# Check for data types and missing values
print(laptop_data.info())



                                                Link  \
0  https://www.flipkart.com/asus-rog-strix-scar-1...   
1  https://www.flipkart.com/asus-rog-strix-scar-1...   
2  https://www.flipkart.com/hp-victus-ryzen-7-oct...   
3  https://www.flipkart.com/lenovo-ideapad-gaming...   
4  https://www.flipkart.com/lenovo-yoga-slim-7-co...   

                                                Name   Brand   Price  \
0  ASUS ROG Strix SCAR 17 Core i9 12th Gen - (32 ...    ASUS  234990   
1  ASUS ROG Strix SCAR 15 Core i9 12th Gen - (32 ...    ASUS  229990   
2  HP Victus Ryzen 7 Octa Core 5800H - (16 GB/512...      HP  104091   
3  Lenovo IdeaPad Gaming 3i Ryzen 7 Octa Core R7-...  Lenovo   87717   
4  Lenovo Yoga Slim 7 Core i5 11th Gen - (16 GB/5...  Lenovo   75990   

                                            Category SSD Capacity    RAM  \
0                                             Gaming         1 TB  32 GB   
1                                             Gaming         1 TB  32 GB   
2 

In [3]:
# Inspect the dataset
print(laptop_data.head())
print(laptop_data.info())



                                                Link  \
0  https://www.flipkart.com/asus-rog-strix-scar-1...   
1  https://www.flipkart.com/asus-rog-strix-scar-1...   
2  https://www.flipkart.com/hp-victus-ryzen-7-oct...   
3  https://www.flipkart.com/lenovo-ideapad-gaming...   
4  https://www.flipkart.com/lenovo-yoga-slim-7-co...   

                                                Name   Brand   Price  \
0  ASUS ROG Strix SCAR 17 Core i9 12th Gen - (32 ...    ASUS  234990   
1  ASUS ROG Strix SCAR 15 Core i9 12th Gen - (32 ...    ASUS  229990   
2  HP Victus Ryzen 7 Octa Core 5800H - (16 GB/512...      HP  104091   
3  Lenovo IdeaPad Gaming 3i Ryzen 7 Octa Core R7-...  Lenovo   87717   
4  Lenovo Yoga Slim 7 Core i5 11th Gen - (16 GB/5...  Lenovo   75990   

                                            Category SSD Capacity    RAM  \
0                                             Gaming         1 TB  32 GB   
1                                             Gaming         1 TB  32 GB   
2 

In [4]:
# Create Price Range column (example: Low, Medium, High)
def categorize_price(price):
    if price < 60000:
        return 'Low'
    elif 60000 <= price < 100000:
        return 'Medium'
    else:
        return 'High'

laptop_data['Price Range'] = laptop_data['Price'].apply(categorize_price)


In [5]:
# Create unique values for dropdowns
unique_brands = laptop_data['Brand'].unique()
unique_ssd_capacities = laptop_data['SSD Capacity'].unique()
unique_ram_sizes = laptop_data['RAM'].unique()

# Display unique values (optional, for verification)
print("Unique Brands:", unique_brands)
print("Unique SSD Capacities:", unique_ssd_capacities)
print("Unique RAM Sizes:", unique_ram_sizes)


Unique Brands: ['ASUS' 'HP' 'Lenovo' 'DELL' 'MSI' 'realme' 'Avita' 'acer' 'SAMSUNG'
 'Infinix' 'LG' 'APPLE' 'Nokia' 'RedmiBook' 'lenovo' 'Mi' 'Vaio']
Unique SSD Capacities: ['1 TB' '512 GB' '256 GB' '128 GB' nan '2 TB']
Unique RAM Sizes: ['32 GB' '16 GB' '4 GB' '8 GB']


In [6]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

# Combine relevant features into a single string for each laptop, handling missing values
laptop_data['combined_features'] = (
    laptop_data['Processor'].fillna('') + ' ' +
    laptop_data['Graphic Processor'].fillna('') + ' ' +
    laptop_data['RAM'].fillna('') + ' ' +
    laptop_data['SSD Capacity'].fillna('') + ' ' +
    laptop_data['Color'].fillna('') + ' ' +
    laptop_data['Brand'].fillna('')
)

# Create the TF-IDF Vectorizer
tfidf_vectorizer = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf_vectorizer.fit_transform(laptop_data['combined_features'])

# Calculate cosine similarity
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

# Reset the index of the DataFrame
laptop_data = laptop_data.reset_index()
indices = pd.Series(laptop_data.index, index=laptop_data['Name'])


In [7]:
def get_recommendations(title, cosine_sim=cosine_sim):
    # Get the index of the laptop that matches the title
    idx = indices[title]

    # Get the pairwise similarity scores of all laptops with that laptop
    sim_scores = list(enumerate(cosine_sim[idx]))

    # Sort the laptops based on the similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Get the scores of the 10 most similar laptops
    sim_scores = sim_scores[1:11]

    # Get the laptop indices
    laptop_indices = [i[0] for i in sim_scores]

    # Return the top 10 most similar laptops
    return laptop_data.iloc[laptop_indices][['Name', 'Brand', 'Price', 'Link', 'combined_features']]


In [8]:
# Prepare the user-item interaction matrix
# Create a DataFrame to hold ratings where 'Name' is the index
ratings_data = laptop_data[['Name', 'User Rating']]

# Set the laptop names as index and fill NaN with 0 (assuming no rating means 0)
user_item_matrix = ratings_data.set_index('Name').fillna(0)

# Display the user-item matrix
print(user_item_matrix.head())



                                                    User Rating
Name                                                           
ASUS ROG Strix SCAR 17 Core i9 12th Gen - (32 G...          5.0
ASUS ROG Strix SCAR 15 Core i9 12th Gen - (32 G...          0.0
HP Victus Ryzen 7 Octa Core 5800H - (16 GB/512 ...          0.0
Lenovo IdeaPad Gaming 3i Ryzen 7 Octa Core R7-5...          0.0
Lenovo Yoga Slim 7 Core i5 11th Gen - (16 GB/51...          0.0


In [9]:
from sklearn.metrics.pairwise import cosine_similarity

# Compute the cosine similarity between laptops based on ratings
collab_sim = cosine_similarity(user_item_matrix)

# Create a DataFrame for easier handling
collab_sim_df = pd.DataFrame(collab_sim, index=user_item_matrix.index, columns=user_item_matrix.index)

# Display the collaborative similarity matrix
print(collab_sim_df.head())


Name                                                ASUS ROG Strix SCAR 17 Core i9 12th Gen - (32 GB/1 TB SSD/Windows 11 Home/8 GB Graphics/NVIDIA GeForce RTX 3070 Ti) G733ZW-LL139WS Gaming Laptop  (17.3 inch, Off Black, 2.90 kg, With MS Office)  \
Name                                                                                                                                                                                                                                                    
ASUS ROG Strix SCAR 17 Core i9 12th Gen - (32 G...                                                1.0                                                                                                                                                   
ASUS ROG Strix SCAR 15 Core i9 12th Gen - (32 G...                                                0.0                                                                                                                                                   
HP V

In [10]:
def collaborative_recommendations(laptop_name, num_recommendations=5):
    # Check if the laptop exists in the ratings data
    if laptop_name not in collab_sim_df.index:
        return f"{laptop_name} not found in ratings data."

    # Get similarity scores for the given laptop
    sim_scores = collab_sim_df[laptop_name].sort_values(ascending=False)

    # Exclude the laptop itself from the recommendations
    sim_scores = sim_scores.drop(laptop_name)

    # Get the top N recommendations
    recommended_laptops = sim_scores.head(num_recommendations)

    # Return the recommended laptop names
    return recommended_laptops.index.tolist()


In [11]:
! pip install streamlit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.9/41.9 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 44.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 73.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.3/79.3 kB 6.6 MB/s eta 0:00:00


In [12]:
! wget -q -O - ipv4.icanhazip.com



35.185.116.118


In [ ]:
! streamlit run laptop_recommendation_app.py & npx localtunnel --port 8501




  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://35.185.116.118:8501

your url is: https://violet-clowns-type.loca.lt
